In [ ]:
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import importlib

import open3d as o3d
import numpy as np
from src import model_save_load
importlib.reload(model_save_load)
# from model_save_load import load_mesh
from src import object_render
importlib.reload(object_render)
# from object_render import get_render
from src import convex_decomposition
importlib.reload(convex_decomposition)
from src import object_detection
importlib.reload(object_detection)
from src import bbox_utils
importlib.reload(bbox_utils)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/be2r-lab-210134/Python/anaconda3/envs/point_cloud/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/be2r-lab-210134/Python/anaconda3/envs/point_cloud/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/be2r-lab-210134/Python/anaconda3/envs/point_cloud/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/67

<module 'src.bbox_utils' from '/home/be2r-lab-210134/Python/functional_grasp/11.04.2025/Affordance/src/bbox_utils.py'>

In [32]:
data_dir = Path("./data/hammer")
mesh_file = data_dir / "object_convex_decomposition.obj"
mesh_o3d, mesh_trimesh, info = model_save_load.load_mesh(mesh_file, vis=False)

In [33]:
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False)
ctr = vis.get_view_control()





In [42]:
camera =  ctr.convert_to_pinhole_camera_parameters()
camera.intrinsic.intrinsic_matrix

array([[935.30743609,   0.        , 959.5       ],
       [  0.        , 935.30743609, 539.5       ],
       [  0.        ,   0.        ,   1.        ]])

In [5]:
ctr.get_field_of_view()

60.0

In [43]:
np.round(camera.extrinsic,1)

array([[  1. ,   0. ,   0. ,  -0. ],
       [ -0. ,  -1. ,  -0. ,   0. ],
       [ -0. ,  -0. ,  -1. , 510.3],
       [  0. ,   0. ,   0. ,   1. ]])

In [44]:
np.round(np.linalg.inv(camera.extrinsic),1)

array([[  1. ,   0. ,   0. ,   0. ],
       [ -0. ,  -1. ,  -0. ,  -0. ],
       [ -0. ,  -0. ,  -1. , 510.3],
       [  0. ,   0. ,   0. ,   1. ]])

In [41]:
ctr.set_lookat([0,0,0])
# ctr.set_front([0,0,-1])

In [36]:
vis.add_geometry(mesh_o3d)


True

In [29]:
camera.extrinsic @ np.array([0,0,0,1]) # world 0 in camera coordinates

array([4.50004578e-01, 8.37596893e-01, 5.09792807e+02, 1.00000000e+00])

In [27]:
camera.extrinsic @ np.array([187.15182419, -77.65317682,  12.37077629, 1])
camera.extrinsic @ np.array([184.91030874, -76.8599313 ,  13.74452546, 1])



array([185.36031332,  77.69752819, 496.04828114,   1.        ])

In [100]:
np.linalg.inv(camera.extrinsic) @ np.array([0,0,0,1])

array([1.54643392e+03, 0.00000000e+00, 1.54643392e+03, 1.00000000e+00])

In [24]:
ctr.set_zoom(3)

In [20]:
vis.capture_depth_point_cloud("pc_world.ply",do_render=True, convert_to_world_coordinate=True)
vis.capture_depth_point_cloud("pc.ply",do_render=True, convert_to_world_coordinate=False)

In [21]:
pc = o3d.io.read_point_cloud("pc.ply")
pc_world = o3d.io.read_point_cloud("pc_world.ply")
o3d.visualization.draw_geometries([pc, pc_world])

In [22]:
np.array(pc_world.points)

array([[184.91030874, -76.8599313 ,  13.74452546],
       [185.03379624, -76.68986965,  14.83025874],
       [185.33823388, -76.59619297,  15.42832351],
       ...,
       [188.6737428 ,  77.29187834,   7.98048871],
       [189.20219668,  77.28862628,   8.0018338 ],
       [190.35416189,  77.53602829,   6.37799561]])

In [23]:
np.array(pc.points)

array([[ 185.35992939,  -77.69736728, -496.0472633 ],
       [ 185.48341664,  -77.52730598, -494.96153228],
       [ 185.78785365,  -77.4336295 , -494.36346874],
       ...,
       [ 189.12335566,   76.45412308, -501.81128812],
       [ 189.65180844,   76.45087103, -501.78994307],
       [ 190.80377127,   76.69827254, -503.4137779 ]])

In [84]:
vis.capture_screen_image('1.jpg',do_render=True)